# Adds
* training/networks.py
    * SignalEncoder()

In [12]:
import torch
import sys
sys.path.append('/Users/menglidaren/Desktop/repo/3DSyn/styleganv2')
from styleganv2.training.depthgan import SignalEncode as SE

angle = torch.tensor(28)
SE_i =SE(frequency=256,w_dim=512)
print(SE_i(angle,torch.randn(3,1,512)).size())

torch.Size([3, 1, 512])


What is the shape of torch.zeros([]) after backwarding?

In [13]:
import torch
class S(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.b = torch.nn.Parameter(torch.zeros([]))
        self.conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=[1,1])

    def forward(self,x):
        y = self.conv(x)
        y = y + self.b
        return y
Model = S()
opt = torch.optim.SGD(Model.parameters(),lr=0.1)
loss = torch.nn.MSELoss()
y = Model(torch.ones([2,1,3,3]))
l = loss(y,torch.zeros([2,1,3,3]))
opt.zero_grad()
l.backward()
opt.step()
print(Model.b.shape)

torch.Size([])


reshape test

In [14]:
import torch
x= torch.arange(1,11)
print(x.reshape(2,5))

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10]])


In [15]:
import contextlib

class simple_with(object):
    def __init__(self):
        print('init')

    def __enter__(self):
        print('enter')

    def __exit__(self,type,value,trace):
        print('exit')

@contextlib.contextmanager
def dec():
    with simple_with():
        yield
    print('exit_dec')

with dec():
    print('test')


init
enter
test
exit
exit_dec


In [4]:
%load_ext autoreload
%autoreload 2
import torch
import styleganv2.training.depthgan as dp
G_rgb = dp.GeneratorRGB(z_dim=512,c_dim=0,w_dim=512,img_resolution=128,img_channels=3)
channels_dict = { res: min(32768 // res, 512) for res in [4,8,16,32,64,128]}
print(channels_dict)
priors = [(torch.randn(3,channels_dict[res],res,res), torch.randn(3,channels_dict[res],res,res)) for res in [4,8,16,32,64,128]]
z = torch.randn(3,512)
print(G_rgb(z,None,priors).size())

{4: 512, 8: 512, 16: 512, 32: 512, 64: 512, 128: 256}


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 4 but got size 8 for tensor number 1 in the list.

In [5]:
# hook test
import torch

class LN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.L1 = torch.nn.Conv2d(in_channels=3,out_channels=6,kernel_size=(2,2))
        self.L2 = torch.nn.Conv2d(in_channels=6,out_channels=9,kernel_size=(2,2))

    def forward(self,x):
        x = self.L1(x)
        x = self.L2(x)
        return x

tmp = []
def hook(module, input, output):
    tmp.append(output)
L = LN()
h = L.L1.register_forward_hook(hook)

x = torch.randn(3,3,5,5)
y = L(x)
h.remove()
print(x.size())
print(tmp[0].size())
print(y.size())

torch.Size([3, 3, 5, 5])
torch.Size([3, 6, 4, 4])
torch.Size([3, 9, 3, 3])
